In this playbook we try to build a index fro searching related wikipages

In [17]:
# Preprocessing of wiki test and generate a title and file pointer json for each file
# This is for future using

import gc
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
stemmer = PorterStemmer()
nonstop = stopwords.words('english')

# text prepprocessing
def helper(name):
    # remove brackets
    name = name.replace("-lrb-", '').replace('-rrb-', '')
    words = []
    # remove punctuation
    name = name.translate(str.maketrans('', '', string.punctuation))
    # tokenize and stem
    for word in nltk.word_tokenize(name):
        word = stemmer.stem(word)
        if word not in nonstop:
            words.append(word)
    return " ".join(words)

def build_doc_list(path: str, str_dict: {}) -> list():
    # open a file and get the file pointer
    cur_file = open(path, "r", encoding='utf-8')
    #file_no = int(path[-7:-4])
    
    line = cur_file.readline()
    
    # normalize the title
    cur_name = line.split()[0] #.lower().replace('_', ' ')
    
    #cap = helper(unicodedata.normalize("NFD", cur_name))
    cap = unicodedata.normalize("NFD", cur_name)
    str_dict[cap] = str_dict.setdefault(cap, [])
    #str_dict[cap][cur_name].append(file_no)
    str_dict[cap].append(0)
    start = 0
    
    while line:
        pointer = cur_file.tell() #returns the location of the next line

        line = cur_file.readline()
        if len(line) > 0:
            tmp = line.split()[0] #.lower().replace('_', ' ')

            # tmp_cap = helper(unicodedata.normalize("NFD", tmp))
            tmp_cap = unicodedata.normalize("NFD", tmp)
            # if a new document
            if cap != tmp_cap:
                # end of the last document

                # here store the start of new one and end of last one
                str_dict[cap].append(pointer-1)
                str_dict[tmp_cap] = str_dict.setdefault(tmp_cap, [])
                #str_dict[tmp_cap][tmp].append(file_no)
                str_dict[tmp_cap].append(pointer)        
                cap = tmp_cap
    str_dict[cap].append(cur_file.tell())
    cur_file.close()
    gc.collect()
    

In [18]:
import json
# parse file into a file name map
def file_parse(path):
    
    test = {}
    build_doc_list(path, test)
    cur_json = open("dict2/" "dict" + str(path[-7:-4]) + ".json", 'w')
    json.dump(test, cur_json)
    cur_json.close()
    




In [19]:
# generate title_doc_map
import os
import unicodedata
path = "wiki-pages-text" 
files= os.listdir(path)
for file in files:
    print(file)
    file_parse("wiki-pages-text/" + file)
            
            


wiki-009.txt
wiki-021.txt
wiki-035.txt
wiki-034.txt
wiki-020.txt
wiki-008.txt
wiki-036.txt
wiki-022.txt
wiki-023.txt
wiki-037.txt
wiki-033.txt
wiki-027.txt
wiki-026.txt
wiki-032.txt
wiki-024.txt
wiki-030.txt
wiki-018.txt
wiki-019.txt
wiki-031.txt
wiki-025.txt
wiki-042.txt
wiki-056.txt
wiki-081.txt
wiki-095.txt
wiki-094.txt
wiki-080.txt
wiki-057.txt
wiki-043.txt
wiki-069.txt
wiki-055.txt
wiki-041.txt
wiki-096.txt
wiki-082.txt
wiki-109.txt
wiki-108.txt
wiki-083.txt
wiki-097.txt
wiki-040.txt
wiki-054.txt
wiki-068.txt
wiki-050.txt
wiki-044.txt
wiki-078.txt
wiki-093.txt
wiki-087.txt
wiki-086.txt
wiki-092.txt
wiki-079.txt
wiki-045.txt
wiki-051.txt
wiki-047.txt
wiki-053.txt
wiki-084.txt
wiki-090.txt
wiki-091.txt
wiki-085.txt
wiki-052.txt
wiki-046.txt
wiki-063.txt
wiki-077.txt
wiki-088.txt
wiki-103.txt
wiki-102.txt
wiki-089.txt
wiki-076.txt
wiki-062.txt
wiki-048.txt
wiki-074.txt
wiki-060.txt
wiki-100.txt
wiki-101.txt
wiki-061.txt
wiki-075.txt
wiki-049.txt
wiki-071.txt
wiki-065.txt
wiki-059.txt

In [ ]:
import nltk
# generate term_title_map
def term_title_map():
    term_map = {}
    # open the json file to get titles
    fp = open("title_doc_map.json", "r")
    title_doc = json.load(fp)
    fp.close()
    gc.collect()
    print("***************")
    for key in title_doc.keys():
        for term in nltk.word_tokenize(key):
            term_map[term] = term_map.setdefault(term, [])
            term_map[term].append(key)
    print("***************")
    del title_doc
    gc.collect()
    cur_file = open("term_title_map.json", "w")
    json.dump(term_map, cur_file)
    cur_file.close()
    gc.collect()
    
term_title_map()
    

In [328]:

fp = open("term_title_map.json", "r")
term = json.load(fp)
fp.close()
gc.collect()


0

In [22]:
# title_doc_map
def get_all_title():
    files = os.listdir("dict")
    tmp = {}
    for file in files:
        print(file)
        # load json file to buid a title:dict_json file
        fp = open("dict/" + file, 'r')
        tmp_dict = json.load(fp)
        keys = tmp_dict.keys()
        for key in keys:
            tmp[key] = tmp.setdefault(key, [])
            file_no = int(file[-8:-5])
            length = len(tmp_dict[key])
            i = 0
            while i < length:
                tmp[key].append(file_no)
                tmp[key].extend(tmp_dict[key][i:i+2])
                i += 2
        # delete the variable and call gc tp prevent overflow
        del tmp_dict
        del keys
        gc.collect()
    cur_file = open("title_doc_map.json", "w")
    json.dump(tmp, cur_file)
    cur_file.close()
    gc.collect()
get_all_title()
        

dict058.json
dict019.json
dict035.json
dict062.json
dict074.json
dict023.json
dict054.json
dict003.json
dict015.json
dict107.json
dict042.json
dict039.json
dict081.json
dict078.json
dict097.json
dict079.json
dict096.json
dict080.json
dict038.json
dict106.json
dict043.json
dict014.json
dict002.json
dict055.json
dict022.json
dict075.json
dict063.json
dict034.json
dict018.json
dict059.json
dict052.json
dict005.json
dict013.json
dict101.json
dict044.json
dict068.json
dict087.json
dict091.json
dict029.json
dict009.json
dict048.json
dict033.json
dict064.json
dict072.json
dict025.json
dict024.json
dict073.json
dict065.json
dict032.json
dict049.json
dict008.json
dict028.json
dict090.json
dict069.json
dict086.json
dict100.json
dict045.json
dict012.json
dict004.json
dict053.json
dict093.json
dict085.json
dict046.json
dict103.json
dict011.json
dict007.json
dict050.json
dict027.json
dict070.json
dict089.json
dict066.json
dict031.json
dict030.json
dict088.json
dict067.json
dict071.json
dict026.json

In [329]:
cur_file = open("title_doc_map.json", "r")
doc = json.load(cur_file)

In [333]:
# testing
a = term["rose"]
print(a)
print(doc[a[0]])

contents = json.load(open("dict/dict0" + str(doc[a[0]][0]) + ".json", 'r'))
tmp = contents[a[0]]

cur_file = open("wiki-pages-text/wiki-0" + str(doc[a[0]][0]) + ".txt", 'br')
cur_file.seek(tmp[0])
s = cur_file.read(tmp[1] - tmp[0])
print(str(s, encoding='utf-8'))

['liebe ist eine rose', 'lieutenant rose films', 'lila rose kaplan', 'list of alister clark roses', 'lee rose -lrb-director-rrb-', 'life goes on -lrb-the desert rose band album-rrb-', 'lianna rose', 'lee rose', 'like a rose -lrb-album-rrb-', 'les roses de mon silence', 'leonard john rose', 'lionel rose', 'life is a bed of roses', 'le spectre de la rose', 'linwood h. rose', 'leonard rose -lrb-hacker-rrb-', 'linlithgow rose f.c.', 'lee rose -lrb-lighting designer-rrb-', 'lee and rose warner nature center', 'leon rose', 'leaton rose', 'liliana rose', 'like red on a rose', 'like a rose', 'camille rose garcia', 'carly rose sonenclar', 'buy me a rose', 'c. brian rose', 'carl rose tenement in bydgoszcz', 'calypso rose', 'canton of marseille-la rose', 'caroline rose isenberg', 'caroline rose hunt', 'canyon rose academy', 'campbell rose', 'carol m. rose', 'gabrielle rose -lrb-actress-rrb-', 'g.s.d. rosignano sei rose', 'gabrielle rose -lrb-swimmer-rrb-', 'evening of roses', 'for love of audrey 